In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## imports
import os
import deepspeed
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import time
import logging
import json
logging.basicConfig(filename='./logs/train-sum-qa-deepspeed.log', level=logging.INFO)
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer, models, util, evaluation, losses, InputExample
from TypingDataset import TypingDataset, MSELoss

[2023-11-08 15:06:02,992] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
## data_paths for NLI

data_folder_path = '/drive02/novak/datasets/downloads/qa/'

data_filenames = [
    # 'gooaq_pairs.jsonl',
    'yahoo_answers_title_answer.jsonl',
    'stackexchange_duplicate_questions_title_title.jsonl',
    'eli5_question_answer.jsonl',
    'yahoo_answers_title_question.jsonl',
    # 'yahoo_answers_question_answer.jsonl',
    # 'wikihow.jsonl',
    # 'NQ-train_pairs.jsonl',
    # 'amazon-qa-train-pairs.jsonl',
    # 'squad_pairs.jsonl',
    # 'quora_duplicates.jsonl',
    # 'PAQ_pairs.jsonl',
]

def load_data(filename):
    
    ret = []
    with open(f'{data_folder_path}{filename}', 'r') as rf:
        
        # lines = rf.readlines()
        
        lines = [InputExample(texts=json.loads(line)) for line in tqdm(rf)]
        
        return lines
        
        # for line in tqdm(lines):
        #     line_data = InputExample(texts=json.loads(line))
        #     ret.append(line_data)
        
    # return ret
            


In [4]:
data = []
for dataset in data_filenames:
    
    data += load_data(dataset)

0it [00:00, ?it/s]

1198260it [00:07, 154837.78it/s]
304525it [00:01, 199283.24it/s]
325475it [00:01, 166403.85it/s]
659896it [00:03, 213817.61it/s]


In [5]:
print(len(data))
pair_data_loader = DataLoader(data, batch_size=300*2, shuffle=True)
# pair_data_loader = TypingDataLoader(data, batch_size=300*4, shuffle=True)

2488156


In [6]:
model_name = 'all-MiniLM-L12-v2'
print('djovak')
# model = SentenceTransformer(model_name, device='cuda:0', )

tokenizer_args = {
  "clean_up_tokenization_spaces": True,
  "cls_token": "[CLS]",
  "do_basic_tokenize": True,
  "do_lower_case": True,
  "mask_token": "[MASK]",
  "max_length": 128,
  "model_max_length": 512,
  "never_split": None,
  "pad_to_multiple_of": None,
  "pad_token": "[PAD]",
  "pad_token_type_id": 0,
  "padding_side": "right",
  "sep_token": "[SEP]",
  "stride": 0,
  "strip_accents": None,
  "tokenize_chinese_chars": True,
  "tokenizer_class": "BertTokenizer",
  "truncation_side": "right",
  "truncation_strategy": "longest_first",
  "unk_token": "[UNK]"
}


djovak


In [7]:
import torch
model_name = 'qa-yotta-L12-v1'
word_embedding_model = models.Transformer('microsoft/MiniLM-L12-H384-uncased', max_seq_length=512, tokenizer_args=tokenizer_args)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=384, activation_function=torch.nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

In [8]:
negative_loss = losses.MultipleNegativesRankingLoss(model=model, similarity_fct=util.cos_sim, scale=80)

In [7]:
import os
print(os.environ['LOCAL_RANK'])

KeyError: 'LOCAL_RANK'

In [9]:
# import os
# print(os.environ['LOCAL_RANK'])

ds_config = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": 0.0002,
            "weight_decay": 0.01
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_num_steps": 200
        }
    },

    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
        "allgather_partitions": True,
        "allgather_bucket_size": 2e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 2e8,
        "contiguous_gradients": True
    },
    
    "comms_logger": {
    "enabled": True,
    "verbose": False,
    "prof_all": True,
    "debug": False
    },

    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
    "train_micro_batch_size_per_gpu": 300
}

engine, optimizer, eng_dataloader, *_ = deepspeed.initialize(
    model=negative_loss,
    config= ds_config,
    training_data=data,
    collate_fn=model.smart_batching_collate,
    
)


[2023-11-08 14:55:20,692] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.12.2, git-hash=unknown, git-branch=unknown
[2023-11-08 14:55:20,694] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-11-08 14:55:20,695] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...


[2023-11-08 14:55:21,288] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=207.189.105.122, master_port=29500
[2023-11-08 14:55:21,290] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2023-11-08 14:55:23,821] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Installed CUDA version 11.2 does not match the version torch was compiled with 11.8 but since the APIs are compatible, accepting this combination
Installed CUDA version 11.2 does not match the version torch was compiled with 11.8 but since the APIs are compatible, accepting this combination


Using /home/novak/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/novak/.cache/torch_extensions/py38_cu118/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load cpu_adam op: 3.6252942085266113 seconds
Adam Optimizer #0 is created with AVX512 arithmetic capability.
Config: alpha=0.000200, betas=(0.900000, 0.999000), weight_decay=0.010000, adam_w=1
[2023-11-08 14:55:30,333] [INFO] [logging.py:96:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adamw as basic optimizer


Loading extension module cpu_adam...


[2023-11-08 14:55:30,335] [INFO] [logging.py:96:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2023-11-08 14:55:30,344] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = DeepSpeedCPUAdam
[2023-11-08 14:55:30,344] [INFO] [utils.py:56:is_zero_supported_optimizer] Checking ZeRO support for optimizer=DeepSpeedCPUAdam type=<class 'deepspeed.ops.adam.cpu_adam.DeepSpeedCPUAdam'>
[2023-11-08 14:55:30,345] [INFO] [logging.py:96:log_dist] [Rank 0] Creating torch.float16 ZeRO stage 2 optimizer
[2023-11-08 14:55:30,346] [INFO] [stage_1_and_2.py:147:__init__] Reduce bucket size 200000000
[2023-11-08 14:55:30,347] [INFO] [stage_1_and_2.py:148:__init__] Allgather bucket size 200000000
[2023-11-08 14:55:30,347] [INFO] [stage_1_and_2.py:149:__init__] CPU Offload: True
[2023-11-08 14:55:30,348] [INFO] [stage_1_and_2.py:150:__init__] Round robin gradient partitioning: False
[2023-11-08 14:55:31,729] [INFO] [utils.py:802:see_memory_usage] Before ini

In [11]:
from tqdm import tqdm
from sentence_transformers.util import batch_to_device
import os
# os.environ['MASTER_ADDR'] = 'localhost'
# os.environ['MASTER_PORT'] = '9994'
# os.environ['RANK'] = "0"
# os.environ['LOCAL_RANK'] = "0"
# os.environ['WORLD_SIZE'] = "1"

# if args.local_rank == -1: 
#     args.local_rank = 0

# 3. finally init deepspeed dist and set the default device
# deepspeed.init_distributed()
# device = torch.device("cuda", 0)

# pair_data_loader.collate_fn = model.smart_batching_collate

for batch in tqdm(eng_dataloader):
    features, labels = batch
    # labels.to(model._target_device)
    # features = list(map(lambda batch: batch_to_device(batch, model._target_device), features))
    loss = engine(features, labels)
    engine.backward(loss)
    engine.step()

  0%|          | 0/8294 [00:00<?, ?it/s]

ERROR:tornado.general:SEND Error: Host unreachable
  0%|          | 0/8294 [06:57<?, ?it/s]


In [ ]:
# try:
#     model.fit(train_objectives=[(pair_data_loader, negative_loss)], 
#             epochs=10,
#             use_amp=True,
#             warmup_steps = int(len(pair_data_loader)*0.1), 
#             weight_decay = 0.01,
#             evaluation_steps=100, 
#             checkpoint_save_total_limit=20,
#             checkpoint_save_steps=100,
#             checkpoint_path = f'./typing-checkpoints/qa-yotta-{model_name}/',
#             output_path=f'./typing/qa-yotta-{model_name}/')
# finally:
#     import torch
#     with torch.no_grad():
#         torch.cuda.empty_cache()

In [ ]:
## TODO

# vizulizacija loss-a na weights and biases
# pozivanje MTAB benchmark-a kao eval
# prelazak pipeline-a na Deepspeed(fp16, optimizovanija paralelizacija)
# vizulatizacija EVAL MTAB metrika, kako rastu tokom treninga
# 

# optimizacija citanja podataka, da ne bude sve u ramu
# MOZDA skupi jos podataka
# MOZDA KVALITET uzimanje iz dataset-a po pravilu iz papira
# MOZDA(low priority) MTAB PARALLEL GPU
